##### Silver Speech and Golden Silence: Spoiler Detection Project

### Neural Network using ULMFiT 

Universal Language Model Fine-tuning (ULMFiT, Howard & Ruder, 2018) is an effective transfer learning method which will be applied to our problem.

This method involves fine-tuning a pre-trained language model (LM), trained on the Wikitext 103 dataset, to a new dataset.

In [1]:
#!pip install fastai

In [2]:
# import libraries
import fastai
from fastai import *
from fastai.text import * 
import pandas as pd
import numpy as np
from functools import partial
import io
import os
import swifter

In [3]:
df_val = pd.read_json('/Users/juliaschafer/NF_Capstone_Spoiler_Detection/data/validation_preprocessed.json')

In [4]:
df_test = pd.read_json('/Users/juliaschafer/NF_Capstone_Spoiler_Detection/data/test_preprocessed.json')

In [5]:
df_trn = pd.read_json('/Users/juliaschafer/NF_Capstone_Spoiler_Detection/data/train_preprocessed.json')

In [6]:
#Join the review texts from list to string
def join_text(df):
    df['text_wo_list'] = df['tokenized'].swifter.apply(lambda x: ' '.join(x))

In [7]:
join_text(df_trn)
join_text(df_val)
join_text(df_test)

In [8]:
#Drop columns not needed
def drop_columns(df):
    df.drop(['user_id', 'time', 'review', 'rating', 'spoiler', 'book_id',
       'review_id', 'genres', 'title', 'description', 'publication_year',
       'publication_month', 'publication_day', 'average_rating',
       'ratings_count', 'num_pages', 'weighted_avg_rating', 'genre',
       'sentence_labels', 'review_texts', 
       'review_len', 'lang', 'prep', 'tokenized'], inplace = True, axis = 1)

In [10]:
drop_columns(df_trn)

drop_columns(df_val)
drop_columns(df_test)

In [14]:
# Separately prepare data for the language model and for the classification model
# Language model data
data_lm = TextLMDataBunch.from_df(train_df = df_trn, valid_df = df_val, path = "")

# Classifier model data
data_clas = TextClasDataBunch.from_df(path = "", train_df = df_trn, valid_df = df_val, vocab=data_lm.train_ds.vocab, bs=32)

/opt/anaconda3/envs/nf/lib/python3.6/site-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(a, dtype=dtype, **kwargs)


/opt/anaconda3/envs/nf/lib/python3.6/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [15]:
#Define the learner object for fine-tuning on pretrained model
learn = language_model_learner(data_lm, arch = AWD_LSTM, pretrained = True, drop_mult=0.7)

In [16]:
# train the learner object with learning rate = 1e-2
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,6.982387,6.772848,0.076972,48:21:10


In [17]:
learn.save_encoder('ft_enc')

In [20]:
learn = text_classifier_learner(data_clas, drop_mult=0.7, arch = AWD_LSTM)
learn.load_encoder('ft_enc');

In [ ]:
learn.fit_one_cycle(1, 1e-2)

In [ ]:
# get predictions
preds, targets = learn.get_preds()

predictions = np.argmax(preds, axis = 1)
pd.crosstab(predictions, targets)

After one epoch, this approach does not seem promising: It takes two days per epoch, delivering bad results. We therefore decide not to waste more time on ULMFiT with more epochs.